In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

Set configuration

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
batch_size = 4
num_workers = 2
epochs = 4
learning_rate = 0.001
PATH = '../data/saved_model/cifar_net.pth'

Crate Dataloader

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root = '../data', train=True,
                                       download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                         shuffle=True, num_workers=num_workers)

testset = torchvision.datasets.CIFAR10(root = '../data', train=False,
                                       download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                        shuffle=False, num_workers=num_workers)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

def imshow(img):
    img = img / 2 + 0.5                        # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show


Files already downloaded and verified
Files already downloaded and verified


Define a Convolutional Neural Network

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net().to(device)


Define a Loss function and optimizer

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate, betas=(0.9, 0.999))


Train the network

In [9]:
for epoch in range(epochs):
    for i, (images, labels) in enumerate(trainloader, 0):
        images = images.to(device)
        labels = labels.to(device)
        
        # forward
        out = net(images)
        loss = criterion(out, labels)

        # zero the parameter gradients
        optimizer.zero_grad()
        # backward
        loss.backward()
        optimizer.step()
        
        if i % 2000 == 1999:
            print("[{epoch}, {index}] loss: {loss}".format(epoch = epoch + 1, index = i + 1, loss = loss.item()))

print("Finished Training")


[1, 2000] loss: 1.729762315750122
[1, 4000] loss: 0.8098723888397217
[1, 6000] loss: 1.2449871301651
[1, 8000] loss: 1.2927680015563965
[1, 10000] loss: 1.1217530965805054
[1, 12000] loss: 1.3344250917434692
[2, 2000] loss: 1.376521348953247
[2, 4000] loss: 0.8653894066810608
[2, 6000] loss: 0.917644202709198
[2, 8000] loss: 0.8173820972442627
[2, 10000] loss: 1.3028066158294678
[2, 12000] loss: 0.5866058468818665
[3, 2000] loss: 2.2376599311828613
[3, 4000] loss: 0.8793030977249146
[3, 6000] loss: 2.238286018371582
[3, 8000] loss: 1.5430010557174683
[3, 10000] loss: 1.2578284740447998
[3, 12000] loss: 1.0302839279174805
[4, 2000] loss: 0.89178466796875
[4, 4000] loss: 1.1796090602874756
[4, 6000] loss: 1.6334439516067505
[4, 8000] loss: 0.7703831195831299
[4, 10000] loss: 0.7971301674842834
[4, 12000] loss: 0.6647030115127563
Finished Training


Save our model

In [10]:
torch.save(net.state_dict(), PATH)

Test the network on the test data

In [13]:
# load network
net = Net().to(device) # the model class
net.load_state_dict(torch.load(PATH))

correct = 0
total = 0

with torch.no_grad():
    for i, (images, labels) in enumerate(testloader, 0):
        images = images.to(device)
        labels = labels.to(device)
        
        out = net(images)
        
        _, predicted = torch.max(out.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 57 %
